In [ ]:
from qiskit import execute, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.circuit.library import QuantumVolume
from dask.distributed import Client

In [ ]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client
from qiskit.providers.aer import AerSimulator

# Initialise your HPC cluster
my_hpc_cluster = LSFCluster(cores=4, memory="30GB")

# Initialise DASK client with the HPC cluster
my_dask_client = Client(my_hpc_cluster)

# Initialise your Aer simulator
backend = AerSimulator()

# Set the client as a custom executor for your backend
backend.set_options(executor=my_dask_client, max_job_size=1)

# Run your circuits
result = backend.run(circ_list).result()

In [ ]:
# Specify molecule details
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.drivers import UnitsType
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver

mol_string='H .0 .0 -2.25; Be .0 .0 .0; H .0 .0 2.25'
max_evals_grouped = 1028

driver = PySCFDriver(atom=mol_string, unit=UnitsType.ANGSTROM, basis='sto3g')

# construct ElectronicStructureProblem with molecule details
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
es_problem = ElectronicStructureProblem(driver)

# Map fermions to qubits
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
qubit_converter = QubitConverter(JordanWignerMapper())

# Add an optimizer
from qiskit.algorithms.optimizers import SLSQP
optimizer = SLSQP(maxiter=5000)

# Construct a quantum instance using DASK Cluster backend
from qiskit.providers.aer import AerSimulator
from qiskit.utils import QuantumInstance
from dask.distributed import Client

exc = Client('your-scheduler-address-here')
simulator = AerSimulator()
simulator.set_options(executor=exc, max_job_size=1)
quantum_instance = QuantumInstance(backend=simulator)

# Initialize a VQE Solver
from qiskit_nature.algorithms import VQEUCCFactory
vqe_solver = VQEUCCFactory(quantum_instance,
                           optimizer=optimizer,
                           include_custom=True,
                           max_evals_grouped=max_evals_grouped)

# Solve for the ground state and print the result
from qiskit_nature.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)
print(res)

# Close the DASK cluster connection once finished
exc.close()